In [1]:
import numpy as np
import torch
import pandas as pd
from torch.autograd import grad
from sklearn.model_selection import train_test_split
import torch.nn as tornn

c:\miniconda\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\miniconda\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\miniconda\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
c:\miniconda\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [10]:
#Initialize weights
class NeuralNetwork:
    def __init__(self,input, output, n_layers, n_neurons, activation_function, loss_function, learning_rate, epoch):
        NeuralNetwork.n_layers = n_layers
        NeuralNetwork.input = input
        NeuralNetwork.output = output
        NeuralNetwork.n_neurons = n_neurons
        NeuralNetwork.activation_function = activation_function
        NeuralNetwork.loss_function = loss_function
        NeuralNetwork.learning_rate = learning_rate
        NeuralNetwork.epoch = epoch
    
    def weight(n_layers, n_neurons):
        W = np.ones((n_layers,n_neurons))
        b = np.ones((n_layers, n_neurons))
        return [W, b]
    
    def softmax(x):
        return np.exp(x) / np.sum(np.exp(x), axis = 0)

    def forward(self,input, n_layers, n_neurons, activation_function):
        network = list(input)
        W = NeuralNetwork.weight(n_layers, n_neurons)[0]
        b = NeuralNetwork.weight(n_layers, n_neurons)[1]
        for i in range(n_layers - 1):
            new_element = activation_function(np.dot(W[i], network[i] + b[i]))
            network.append(new_element)
        network.append(NeuralNetwork.softmax(network[-1]))
        return network
    
    def backward(self, input, output, activation_function, loss_function, n_layers, n_neurons, network):
        #L = loss_function(loss_function)
        #grad_loss = grad(loss_function)
        W = NeuralNetwork.weight(n_layers, n_neurons)[0]
        b = NeuralNetwork.weight(n_layers, n_neurons)[1]
        #print(W)
        #print(list(NeuralNetwork.input))
        #print(loss_function(list(NeuralNetwork.output), network[-1]))
        l = NeuralNetwork.learning_rate
        #W = torch.tensor(W, requires_grad=True)
        #array_input = np.array(NeuralNetwork.input)
        #array_input = array_input.astype(np.float32)
        #inpt = torch.tensor(array_input, requires_grad=True)
        losses = torch.tensor(loss_function(list(NeuralNetwork.output), network[-1]), requires_grad=True)
        print(losses)
        print(W)
        #print(inpt)
        #for i in range(NeuralNetwork.epoch):
            #for j in range(len(W[0])):
                #torch.autograd.backward(loss)
                # Update weights
                #with torch.no_grad():
                    #w -= learning_rate * w.grad
                    #b -= learning_rate * b.grad
                    #W[j] = [weight - l * grad_param for weight, grad_param in zip(W[j], W[j].grad)]
                #gradients = grad(losses,W[j], inpt, , allow_unused=True)
                #print(gradients)

        network = self.forward(input, n_layers, n_neurons, activation_function)
        gradients = [2 * (network[-1] - output) * self.activation_function(network[-1], derivative = True)]

        for i in range(self.n_layers - 1, 0, -1):
            delta = np.dot(gradients[-1], self.weights[i].T) * self.activation_function(network[i], derivative=True)
            gradients.append(delta)

        gradients.reverse()

        for i in range(self.n_layers):
            W[i] -= self.learning_rate * np.dot(network[i].T, gradients[i])
            self.biases[i] -= self.learning_rate * np.sum(gradients[i], axis=0)
                
        return W


In [11]:
def activation(x, derivative = False):
    if derivative:
        return np.sigmoid(x) * (1 - np.sigmoid(x))
    return 1 / (1 + np.exp(-x))

def loss(x,y):
        return 0.5 * (x - y) ** 2

np.vectorize(activation)
np.vectorize(loss)

In [12]:
# Load the data
data = pd.read_excel("Raisin_Dataset.xlsx")

In [13]:
#Transform Class column to 0 and 1 column : Kecimen = 1 and Besni = 0
data["Class"] = np.where(data["Class"] == 'Kecimen',1,0)

# Split the data into training and test sets
X = data.drop(['Class'], axis=1)
y = data['Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_2 = X_train["Area"][:10]
y_train_2 = y_train[:10]

In [14]:
print(np.array(X_train_2))

[ 80437  72483  85739 182788  62835 109254  61492 136340  55746  59902]


In [15]:
nn = NeuralNetwork(X_train_2, y_train_2, n_layers = 3, n_neurons = 10, activation_function = activation, loss_function = loss, learning_rate = 0.1, epoch = 10)

In [16]:
network = nn.forward(X_train_2, n_layers = 3, n_neurons = 10, activation_function = activation)

In [17]:
nn.backward(input = X_train_2, activation_function = activation, output = y_train_2,loss_function = loss, n_layers = 3, n_neurons = 10, network = network)

tensor([0.0000, 0.0000, 0.0000, 0.5000, 0.0000, 0.5000, 0.0000, 0.0000, 0.0000,
        0.0000], dtype=torch.float64, requires_grad=True)
[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]


TypeError: activation() takes from 1 to 2 positional arguments but 3 were given